In [6]:
import rinna_utils

In [12]:
ls ../models

lora-rinna-3.6b/  rinna3.6b/


In [13]:
models_path = '../models'

In [16]:
tokenizer = rinna_utils.load_tokenizer(f'{models_path}/rinna3.6b')

In [17]:
roles = ['ユーザー: ', 'システム: ']

In [41]:
prompt_list = ['会話をしましょう。まずはあなたの自己紹介をお願いします。設定は以下のとおりです。「大学院生, 情報工学専攻, 画像処理が専門」']

In [42]:
prompt = [f"{roles[(i)%2]}{p}" for i, p in enumerate(prompt_list)]
prompt:str = '<NL>'.join(prompt)
prompt += '<NL>システム: '

In [43]:
print(rinna_utils.generate(prompt, tokenizer, model, max_new_tokens=150, output_only_generated=False, escape_special_token=False))

ユーザー: 会話をしましょう。まずはあなたの自己紹介をお願いします。設定は以下のとおりです。「大学院生, 情報工学専攻, 画像処理が専門」<NL>システム: 大学院の画像認識に関する研究をしています。 趣味は、スキーやゴルフです。 (研究テーマ:画像の認識) は、画像内の物体の場所を推定するのに役立ちます。また、研究の重要な部分は、認識技術の様々な分野の知識の習得です. (技術テーマ)は研究分野であり、技術的な知識を習得します. 研究に重要な部分であります.また,研究の中心的な部分は研究ですが、重要な部分を習得しています。研究は重要であり,重要な研究の部分です! :).(研究内容)研究では、高度な画像技術を様々な分野で習得することができますが,重要です,中心的な部分です(技術)。(高度な技術分野)の研究は,


In [44]:
exit_code = ['exit', 'quit']

In [47]:
def gen_prompt(prompt):
    prompt:str = '<NL>'.join(prompt)
    prompt += '<NL>システム: '
    return prompt

In [53]:
prompt = []
while True:
    input_text = input('ユーザー > ')
    if input_text in exit_code:
        break
    prompt.append(f'ユーザー: {input_text}')
    output_text = rinna_utils.generate(
        gen_prompt(prompt), 
        tokenizer, 
        model, 
        max_new_tokens=150, 
        output_only_generated=True, 
        escape_special_token=False)
    print(output_text)
    prompt.append(f'システム: {output_text}')

ユーザー > 何かやりたいことはありますか？
もちろんです。 まず、花や木の剪定点に立って、地面に足跡をつけ、そして、剪りーをし、その後、足音のする方向に向かって歩き、音がする方向に歩き続けることです。そして最後に、すべての方向を回り続けることを約束します。これらの言葉は、自然に流れるような動きを約束しています。また、これらの動きは自然な流れをもたらしてくれるでしょう。最後には約束されています:) これらのことを、動き続けること、約束された方向に向かうことを続けることで、最後にはすべてが自然の流れをたっぷり、また約束されている方向に流れることを誓うために、すべてが約束されて、流れ続けるために約束されることになります。</s>
ユーザー > いいね。私と何して遊びたい？
素晴らしいですね、一緒にゲームをしたり、散歩したりして、お互いの素晴らしいところを共有したいです! 散歩や、公園での運動、海での水泳や釣り、スキーやスケート、サイクリングやスキーなど、素晴らしい自然のアクティビティを一緒に共有することができます!一緒に自然の中でリラックスした時間を過ごすことができますよ。一緒に散歩をしたい場合は、時間と場所を指定してください。散歩に出かける場合は散歩に行き、運動に出かけますので、その時に一緒にいることができ、楽しい時間を過ごしたいられますように、周りに気づかれることなく、自分の素晴らしいことを共有できますように努めますが、お気の毒な場合はお知らせくださいませ。素晴らしい散歩と運動を楽しみにして
ユーザー > exit


In [54]:
prompt

['ユーザー: 何かやりたいことはありますか？',
 'システム: もちろんです。 まず、花や木の剪定点に立って、地面に足跡をつけ、そして、剪りーをし、その後、足音のする方向に向かって歩き、音がする方向に歩き続けることです。そして最後に、すべての方向を回り続けることを約束します。これらの言葉は、自然に流れるような動きを約束しています。また、これらの動きは自然な流れをもたらしてくれるでしょう。最後には約束されています:) これらのことを、動き続けること、約束された方向に向かうことを続けることで、最後にはすべてが自然の流れをたっぷり、また約束されている方向に流れることを誓うために、すべてが約束されて、流れ続けるために約束されることになります。</s>',
 'ユーザー: いいね。私と何して遊びたい？',
 'システム: 素晴らしいですね、一緒にゲームをしたり、散歩したりして、お互いの素晴らしいところを共有したいです! 散歩や、公園での運動、海での水泳や釣り、スキーやスケート、サイクリングやスキーなど、素晴らしい自然のアクティビティを一緒に共有することができます!一緒に自然の中でリラックスした時間を過ごすことができますよ。一緒に散歩をしたい場合は、時間と場所を指定してください。散歩に出かける場合は散歩に行き、運動に出かけますので、その時に一緒にいることができ、楽しい時間を過ごしたいられますように、周りに気づかれることなく、自分の素晴らしいことを共有できますように努めますが、お気の毒な場合はお知らせくださいませ。素晴らしい散歩と運動を楽しみにして']